In [1]:
if 'google.colab' in str(get_ipython()):
    from google.colab import userdata
    access_token = userdata.get('DEFORMER_TOKEN')
    !pip install git+https://$access_token@github.com/ay94/deformer-extractor.git@error-handling

  Cloning https://****@github.com/ay94/deformer-extractor.git (to revision error-handling) to /tmp/pip-req-build-p7n90g7g
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/ay94/deformer-extractor.git' /tmp/pip-req-build-p7n90g7g
  Running command git checkout -b error-handling --track origin/error-handling
  Switched to a new branch 'error-handling'
  Branch 'error-handling' set up to track remote branch 'error-handling' from 'origin'.
  Resolved https://****@github.com/ay94/deformer-extractor.git to commit 93e8061609d7b196343441c7ee60d8af7b114538
  Preparing metadata (setup.py) ... done


In [2]:
from experiment_utils import colab
from experiment_utils.general_utils import FileHandler

# Training

In [ ]:

base_folder = colab.init('My Drive')
config_path = base_folder / 'Final Year Experiments/Thesis-Experiments/scripts'
data_path = base_folder / 'Final Year Experiments/Thesis-Experiments/ExperimentData'
fh = FileHandler(config_path)
data_fh = FileHandler(data_path)

2024-07-18 00:15:05 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ay227/Desktop/Final-Year/Thesis-Experiments/Data-Extraction-Phase/notebooks/My Drive
2024-07-18 00:15:05 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com


In [ ]:

from experiment_utils.train import FineTuneConfig, TCDataset
from experiment_utils.tokenization import TokenStrategyFactory


# Example usage
config_values = fh.load_yaml('train_config.yaml')
tokenization_config = fh.load_yaml('tokenization_config.yaml')
config = FineTuneConfig()


ImportError: cannot import name 'FineTuneConfig' from 'experiment_utils.train' (/Users/ay227/Desktop/Final-Year/Thesis-Experiments/Data-Extraction-Phase/deformer-extractor/experiment_utils/train.py)

In [ ]:
data  = data_fh.load_json('corpora.json')
NERData = data['ANERCorp_CamelLab']

In [ ]:
dataset[1]['input_ids'][:36]

In [ ]:
dataset[1]['labels'][:36]

In [ ]:
model = TCModel(num_tag=10, path='bert-base-uncased')


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizerFast, BertForTokenClassification, BertConfig
from torch.nn.utils.rnn import pad_sequence

# Define a custom dataset
class NERDataset(Dataset):
    def __init__(self, tokenizer, texts, tags, label_map):
        self.tokenizer = tokenizer
        self.texts = texts
        self.tags = tags
        self.label_map = label_map

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        tokens = self.tokenizer.encode_plus(
            self.texts[idx],
            max_length=128,
            truncation=True,
            padding='max_length',
            return_tensors="pt",
            is_split_into_words=True
        )
        labels = [self.label_map[tag] for tag in self.tags[idx]]
        labels = torch.tensor(labels + [self.label_map['O']] * (128 - len(labels)))  # Pad labels
        return tokens['input_ids'].squeeze(0), tokens['attention_mask'].squeeze(0), labels

# Dummy data
texts = [
    ["Hello", "world", "this", "is", "BERT"],
    ["Another", "sentence"]
]
tags = [
    ["O", "O", "O", "O", "B-PER"],
    ["O", "O"]
]
label_map = {"O": 0, "B-PER": 1, "I-PER": 2}

# Initialize tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Initialize dataset
dataset = NERDataset(tokenizer, texts, tags, label_map)
loader = DataLoader(dataset, batch_size=2)


In [ ]:
input_ids.shape[1]

In [ ]:
# Initial model output without hidden states and attentions
for batch in loader:
    input_ids, attention_mask, labels = batch
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)


In [ ]:
model.enable_hidden_states()

In [ ]:
# Initial model output without hidden states and attentions
for batch in loader:
    input_ids, attention_mask, labels = batch
    token_type_ids = torch.tensor([0] * input_ids.shape[1], dtype=torch.long)
    outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, labels=labels)


In [ ]:
torch.save(model.state_dict(), 'model_state_dict.pt')


In [ ]:
loaded_model = TCModel(num_tag=10, path='bert-base-uncased')

loaded_model.load_state_dict(torch.load('model_state_dict.pt'))

In [ ]:
# Initial model output without hidden states and attentions
for batch in loader:
    input_ids, attention_mask, labels = batch
    token_type_ids = torch.tensor([0] * input_ids.shape[1], dtype=torch.long)
    outputs = loaded_model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, labels=labels)


In [ ]:
loaded_model.enable_attentions()

In [ ]:
# Initial model output without hidden states and attentions
for batch in loader:
    input_ids, attention_mask, labels = batch
    token_type_ids = torch.tensor([0] * input_ids.shape[1], dtype=torch.long)
    outputs = loaded_model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, labels=labels)


In [ ]:
outputs.keys()

# Results handling

In [ ]:
base_folder = colab.init('My Drive')
config_path = base_folder / 'Final Year Experiments/Class Imbalance/1_fineTuning'
fh = FileHandler(config_path)

Mounted at /content/drive


In [ ]:
model_name='arabertv02'
data_name='ANERCorp_CamelLab'

In [ ]:
import torch
from tqdm.autonotebook import tqdm
model_outputs = fh.load_pickle(
            f"evalOutputs/{model_name}_{data_name}_regular_outputs.pkl"
        )

# load_model_path = fh.file_path / f"trainOutputs/{model_name}_{data_name}_regular.bin"
# model = torch.load(load_model_path)

In [ ]:
data_loader = model_outputs.test_dataloader
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

In [ ]:
corpora.keys()

dict_keys(['ANERCorp_CamelLab', 'ANERCorp_CamelLab-validation', 'conll2003'])

In [ ]:
data_manager = DatasetManager(corpora, 'ANERCorp_CamelLab-validation', tokenization_config)


In [ ]:
dataset = data_manager.get_dataset('test')

INFO:root:Loading Preprocessor: aubmindlab/bert-base-arabertv02
INFO:root:Loading Tokenizer: aubmindlab/bert-base-arabertv02
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
data_loader = data_manager.get_dataloader('test', 8)

INFO:root:Loading Preprocessor: aubmindlab/bert-base-arabertv02
INFO:root:Loading Tokenizer: aubmindlab/bert-base-arabertv02


In [ ]:
analysis='attention_mask'
for i, (data_old, data_new) in enumerate(zip(model_outputs.test_dataloader, data_loader)):
    tensor_old = data_old[analysis][data_old[analysis]!=0][:49]
    tensor_new = data_new[analysis][data_new[analysis]!=0][:49]
    differences = tensor_old != tensor_new
    diff_indices = torch.where(differences)[0]
    diff_values_old = tensor_old[diff_indices]
    diff_values_new = tensor_new[diff_indices]

    print(f"Batch {i+1} in Old Setup:", tensor_old)
    print(f"Batch {i+1} in New Setup:", tensor_new)
    print(f"Batch {i+1} in Old Setup Difference:", diff_values_old)
    print(f"Batch {i+1} in New Setup Difference:", diff_values_new)
    if i == 0:  # Just checking the first 3 batches for discrepancies
        break


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Batch 1 in Old Setup: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1])
Batch 1 in New Setup: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1])
Batch 1 in Old Setup Difference: tensor([], dtype=torch.int64)
Batch 1 in New Setup Difference: tensor([], dtype=torch.int64)


In [ ]:
vocab_map[6465]

'بالان'

In [ ]:
dataset.preprocessor.preprocess('أمر')

'أمر'

In [ ]:
dataset.tokenizer.tokenize(dataset.preprocessor.preprocess('أمر'))

['امر']

In [ ]:
dataset.tokenizer('أمر')

{'input_ids': [2, 6177, 3], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

In [ ]:
vocab_map = {v : k for k, v in datset.tokenizer.vocab.items()}

In [ ]:
vocab_map

{tensor([9], device='cuda:0'): False}

In [ ]:
model.eval()
with torch.no_grad():
	final_loss = 0
	preds = None
	labels = None
	for data in tqdm(data_loader, total=len(data_loader)):
		data['words_ids'] = torch.randint(0, 1000, (data['input_ids'].shape[0], data['input_ids'].shape[1])).to(device)
		data['sentence_num'] = torch.randint(0, 10, (data['input_ids'].shape[0],)).to(device)

		for k, v in data.items():
			data[k] = v.to(device)
		outputs = model(**data)
		loss = outputs["average_loss"]
		logits = outputs["logits"]
		final_loss += loss.item()
		if logits is not None:
			preds = (
				logits if preds is None else torch.cat((preds, logits), dim=0)
			)
		if data["labels"] is not None:
			labels = (
				data["labels"]
				if labels is None
				else torch.cat((labels, data["labels"]), dim=0)
			)
	preds = preds.detach().cpu().numpy()
	labels = labels.cpu().numpy()
	# evaluation = Evaluation(labels, preds, inv_labels, final_loss)
	# metrics = evaluation.compute_metrics()

  0%|          | 0/121 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
from experiment_utils.train import Evaluation, Metrics

evaluator = Evaluation(model_outputs.data['inv_labels'], labels, preds, final_loss)
results = evaluator.generate_results()
metrics = Metrics.from_dict(results)
metrics.entity_report

,Tag,Precision,Recall,F1,Support
0,LOC,0.8917749,0.9142012,0.9028488,676
1,MISC,0.6872038,0.5967078,0.6387665,243
2,ORG,0.7814727,0.7167756,0.7477273,459
3,PERS,0.8507973,0.8254144,0.8379136,905
4,micro,0.8347708,0.8055191,0.8198841,2283
5,macro,0.8028122,0.7632747,0.7818141,2283
6,weighted,0.8315803,0.8055191,0.8178120,2283


In [ ]:
from experiment_utils.train import Evaluation, Metrics

evaluator = Evaluation(model_outputs.data['inv_labels'], labels, preds, final_loss)
results = evaluator.generate_results()
metrics = Metrics.from_dict(results)
metrics.entity_report

,Tag,Precision,Recall,F1,Support
0,LOC,0.8917749,0.9142012,0.9028488,676
1,MISC,0.6872038,0.5967078,0.6387665,243
2,ORG,0.7814727,0.7167756,0.7477273,459
3,PERS,0.8507973,0.8254144,0.8379136,905
4,micro,0.8347708,0.8055191,0.8198841,2283
5,macro,0.8028122,0.7632747,0.7818141,2283
6,weighted,0.8315803,0.8055191,0.8178120,2283


In [ ]:
from experiment_utils.train import Evaluation, Metrics

evaluator = Evaluation(model_outputs.data['inv_labels'], labels, preds, final_loss)
results = evaluator.generate_results()
metrics = Metrics.from_dict(results)
metrics.entity_report

,Tag,Precision,Recall,F1,Support
0,LOC,0.8923513,0.9319527,0.9117221,676
1,MISC,0.7475248,0.6213992,0.6786517,243
2,ORG,0.7816901,0.7254902,0.7525424,459
3,PERS,0.8850174,0.8419890,0.8629672,905
4,micro,0.8546697,0.8217258,0.8378741,2283
5,macro,0.8266459,0.7802077,0.8014708,2283
6,weighted,0.8517803,0.8217258,0.8355842,2283


In [ ]:
model.eval()
total_loss = 0
preds = None
labels = None
progress_bar = tqdm(data_loader, total=len(data_loader), desc="Evaluating")

with torch.no_grad():
    for batch_idx, data in enumerate(progress_bar):

        # Move data to the appropriate device
        data = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in data.items()}

        outputs = model(**data)
        loss = outputs["average_loss"]
        logits = outputs["logits"]
        total_loss += loss.item()

        if logits is not None:
            preds = logits if preds is None else torch.cat((preds, logits), dim=0)
        if data["labels"] is not None:
            labels = data["labels"] if labels is None else torch.cat((labels, data["labels"]), dim=0)

        # Set loss for the current batch in the progress bar
        progress_bar.set_postfix({'loss': f"{loss.item():.4f}"})

        # Detailed logging for each logging step
        if (batch_idx + 1) % args.logging_step == 0:
            logging.info("Batch %d/%d - Loss: %.4f", batch_idx + 1, len(data_loader), loss.item())

    preds = preds.detach().cpu().numpy()
    labels = labels.cpu().numpy()
    average_loss = total_loss / len(data_loader)
    logging.info("Evaluation completed. Average Loss per Batch: %.4f", average_loss)

Evaluating:   0%|          | 0/121 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
INFO:root:Batch 50/121 - Loss: 0.0521
INFO:root:Batch 100/121 - Loss: 0.1393
INFO:root:Evaluation completed. Average Loss per Batch: 0.1591


In [ ]:
from experiment_utils.train import Evaluation, Metrics

evaluator = Evaluation(model_outputs.data['inv_labels'], labels, preds, final_loss)
results = evaluator.generate_results()
metrics = Metrics.from_dict(results)


In [ ]:
metrics.entity_report

,Tag,Precision,Recall,F1,Support
0,LOC,0.8923513,0.9319527,0.9117221,676
1,MISC,0.7475248,0.6213992,0.6786517,243
2,ORG,0.7816901,0.7254902,0.7525424,459
3,PERS,0.8850174,0.8419890,0.8629672,905
4,micro,0.8546697,0.8217258,0.8378741,2283
5,macro,0.8266459,0.7802077,0.8014708,2283
6,weighted,0.8517803,0.8217258,0.8355842,2283


# Trainer

In [4]:
from experiment_utils import colab
from experiment_utils.general_utils import FileHandler

base_folder = colab.init('My Drive')
corpora_path = base_folder / 'Final Year Experiments/Thesis-Experiments/ExperimentData'
corpora_fh = FileHandler(corpora_path)
corpora  = corpora_fh.load_json('corpora.json')

config = base_folder / 'Final Year Experiments/Thesis-Experiments/scripts'
config_fh = FileHandler(config)
config = config_fh.load_yaml('config.yaml')


Mounted at /content/drive


In [6]:

import logging
import torch

from experiment_utils.train import TCModel, TCDataset, FineTuneUtils





In [7]:
finetune_config = FineTuneConfig.from_dict(config.get('finetuning'))
tokenization_config = TokenizationConfig.from_dict(config.get('tokenization'))
model_config = ModelConfig.from_dict(config.get('model'))

INFO:root:Configuration validated successfully


In [44]:
data_manager = DatasetManager(corpora, 'ANERCorp_CamelLab', tokenization_config)
num_tags = len(data_manager.corpus['labels'])
model_manager = ModelManager(num_tags, model_config)
optimizer_scheduler_manager = OptimizerSchedulerManager(model_manager.get_model_parameters(), len(data_manager.get_dataset('train')), finetune_config)
args = TrainingArguments.from_config(finetune_config)



INFO:root:Loading BERT Model from: aubmindlab/bert-base-arabertv02
INFO:root:Model aubmindlab/bert-base-arabertv02 loaded and sent to cuda
INFO:root:Loading Preprocessor: aubmindlab/bert-base-arabertv02
INFO:root:Loading Tokenizer: aubmindlab/bert-base-arabertv02


In [26]:
from tqdm.autonotebook import tqdm
import time
from torch.utils.data import DataLoader


In [45]:
dataloader = train_dataloader = data_manager.get_dataloader('train', finetune_config.train_batch_size)
num_train_steps = int(
            (len(dataloader.dataset) / (finetune_config.train_batch_size * finetune_config.accumulation_steps)) * finetune_config.epochs
        )
num_train_steps

INFO:root:Loading Preprocessor: aubmindlab/bert-base-arabertv02
INFO:root:Loading Tokenizer: aubmindlab/bert-base-arabertv02


1037

In [36]:
dataset = torch.load(corpora_fh.file_path/ 'train.pth')
num_train_steps = int(
            (len(dataset) / (finetune_config.train_batch_size * finetune_config.accumulation_steps)) * finetune_config.epochs
        )
num_train_steps

850

In [43]:
len(dataset)

3402

In [41]:
per_e * finetune_config.epochs

850.5

In [27]:
from experiment_utils.train import Evaluation, Metrics


In [28]:
trainer = Trainer(FineTuneUtils, data_manager, model_manager, finetune_config)

INFO:root:Loading BERT Model from: aubmindlab/bert-base-arabertv02
INFO:root:Model aubmindlab/bert-base-arabertv02 loaded and sent to cuda
INFO:root:Loading Preprocessor: aubmindlab/bert-base-arabertv02
INFO:root:Loading Tokenizer: aubmindlab/bert-base-arabertv02
INFO:root:Loading Preprocessor: aubmindlab/bert-base-arabertv02
INFO:root:Loading Tokenizer: aubmindlab/bert-base-arabertv02
INFO:root:num_train_steps:829


In [29]:
metrics = trainer.training_loop()

INFO:root:Start Training Epoch: 1


  0%|          | 0/208 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [24]:
metrics = trainer.training_loop()

INFO:root:Start Training Epoch: 1


  0%|          | 0/213 [00:00<?, ?it/s]

INFO:root:Start Evaluation


  0%|          | 0/121 [00:00<?, ?it/s]

INFO:root:Training Loss: 0.3253120788123826 | Eval Loss: 0.14868087398103816
INFO:root:Epoch completed in 75.20607089996338 s
INFO:root:
Token-Level Evaluation Metrics:
INFO:root:╒════════════════╤═════════════╤══════════╤══════════╤═══════════╕
│ Tag            │   Precision │   Recall │       F1 │   Support │
╞════════════════╪═════════════╪══════════╪══════════╪═══════════╡
│ B-LOC          │    0.887187 │ 0.953593 │ 0.919192 │       668 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ B-MISC         │    0.798942 │ 0.642553 │ 0.712264 │       235 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ B-ORG          │    0.742394 │ 0.813333 │ 0.776246 │       450 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ B-PERS         │    0.891124 │ 0.877622 │ 0.884322 │       858 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ I-LOC          │    0.85     │ 0.819277 │ 0.834356 │        83 │
├────────────────

...

INFO:root:Training Loss: 0.02494110449046736 | Eval Loss: 0.15381122180568593
INFO:root:Epoch completed in 74.81893301010132 s
INFO:root:
Token-Level Evaluation Metrics:
INFO:root:╒════════════════╤═════════════╤══════════╤══════════╤═══════════╕
│ Tag            │   Precision │   Recall │       F1 │   Support │
╞════════════════╪═════════════╪══════════╪══════════╪═══════════╡
│ B-LOC          │    0.906609 │ 0.944611 │ 0.92522  │       668 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ B-MISC         │    0.81383  │ 0.651064 │ 0.723404 │       235 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ B-ORG          │    0.837321 │ 0.777778 │ 0.806452 │       450 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ B-PERS         │    0.880046 │ 0.889277 │ 0.884638 │       858 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ I-LOC          │    0.804598 │ 0.843373 │ 0.823529 │        83 │
├───────────────

  0%|          | 0/213 [00:00<?, ?it/s]

INFO:root:Start Evaluation


  0%|          | 0/121 [00:00<?, ?it/s]

INFO:root:Training Loss: 0.015591064292583352 | Eval Loss: 0.16445452485556813
INFO:root:Epoch completed in 74.85125350952148 s
INFO:root:
Token-Level Evaluation Metrics:
INFO:root:╒════════════════╤═════════════╤══════════╤══════════╤═══════════╕
│ Tag            │   Precision │   Recall │       F1 │   Support │
╞════════════════╪═════════════╪══════════╪══════════╪═══════════╡
│ B-LOC          │    0.900709 │ 0.950599 │ 0.924982 │       668 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ B-MISC         │    0.814433 │ 0.67234  │ 0.736597 │       235 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ B-ORG          │    0.814118 │ 0.768889 │ 0.790857 │       450 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ B-PERS         │    0.883529 │ 0.875291 │ 0.879391 │       858 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ I-LOC          │    0.833333 │ 0.843373 │ 0.838323 │        83 │
├──────────────

In [20]:
metrics = trainer.training_loop()

INFO:root:Start Training Epoch: 1


  0%|          | 0/426 [00:00<?, ?it/s]

INFO:root:Start Evaluation


  0%|          | 0/121 [00:00<?, ?it/s]

INFO:root:Training Loss: 0.18919663876431728 | Eval Loss: 0.15329910497650628
INFO:root:Epoch completed in 88.09396600723267 s
INFO:root:Start Training Epoch: 2


...

  0%|          | 0/426 [00:00<?, ?it/s]

INFO:root:Start Evaluation


  0%|          | 0/121 [00:00<?, ?it/s]

INFO:root:Training Loss: 0.024462520822192906 | Eval Loss: 0.15128497326985185
INFO:root:Epoch completed in 86.73906326293945 s



Token-Level Evaluation Metrics:
╒════════════════╤═════════════╤══════════╤══════════╤═══════════╕
│ Tag            │   Precision │   Recall │       F1 │   Support │
╞════════════════╪═════════════╪══════════╪══════════╪═══════════╡
│ B-LOC          │    0.906475 │ 0.943114 │ 0.924431 │       668 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ B-MISC         │    0.849711 │ 0.625532 │ 0.720588 │       235 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ B-ORG          │    0.858209 │ 0.766667 │ 0.809859 │       450 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ B-PERS         │    0.87456  │ 0.869464 │ 0.872005 │       858 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ I-LOC          │    0.823529 │ 0.843373 │ 0.833333 │        83 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ I-MISC         │    0.846154 │ 0.4      │ 0.54321  │       165 │
├────────────────┼───────────

In [ ]:
metrics = trainer.training_loop()

INFO:root:Start Training Epoch: 1


  0%|          | 0/213 [00:00<?, ?it/s]

INFO:root:Start Evaluation


  0%|          | 0/121 [00:00<?, ?it/s]

INFO:root:Training Loss: 0.284244428384402 | Eval Loss: 0.14134191516555908
INFO:root:Epoch completed in 86.79596447944641 s
INFO:root:Start Training Epoch: 2


...

  0%|          | 0/213 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
INFO:root:Start Evaluation


  0%|          | 0/121 [00:00<?, ?it/s]

INFO:root:Training Loss: 0.015116657487864435 | Eval Loss: 0.1569162806138119
INFO:root:Epoch completed in 86.70140314102173 s



Token-Level Evaluation Metrics:
╒════════════════╤═════════════╤══════════╤══════════╤═══════════╕
│ Tag            │   Precision │   Recall │       F1 │   Support │
╞════════════════╪═════════════╪══════════╪══════════╪═══════════╡
│ B-LOC          │    0.905849 │ 0.950599 │ 0.927684 │       668 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ B-MISC         │    0.836842 │ 0.676596 │ 0.748235 │       235 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ B-ORG          │    0.845411 │ 0.777778 │ 0.810185 │       450 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ B-PERS         │    0.879133 │ 0.898601 │ 0.888761 │       858 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ I-LOC          │    0.811765 │ 0.831325 │ 0.821429 │        83 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ I-MISC         │    0.842857 │ 0.357576 │ 0.502128 │       165 │
├────────────────┼───────────

In [ ]:
metrics = trainer.training_loop()


INFO:root:Start Training Epoch: 1
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Training:   0%|          | 0/213 [00:00<?, ?it/s]

INFO:root:Batch 50/213 - Loss: 0.3316
INFO:root:Batch 100/213 - Loss: 0.1075
INFO:root:Batch 150/213 - Loss: 0.0512
INFO:root:Batch 200/213 - Loss: 0.0734
INFO:root:Training completed. Average Loss: 0.2965
INFO:root:Start Evaluation


Evaluating:   0%|          | 0/121 [00:00<?, ?it/s]

INFO:root:Batch 50/121 - Loss: 0.0498
INFO:root:Batch 100/121 - Loss: 0.0874
INFO:root:Evaluation completed. Average Loss per Batch: 0.1406
INFO:root:Training Loss: 0.29650286274214444 | Eval Loss: 0.14058796456083655
INFO:root:Epoch completed in 88.21844482421875 s
INFO:root:Start Training Epoch: 2


...

Training:   0%|          | 0/213 [00:00<?, ?it/s]

INFO:root:Batch 50/213 - Loss: 0.0108
INFO:root:Batch 100/213 - Loss: 0.0136
INFO:root:Batch 150/213 - Loss: 0.0094
INFO:root:Batch 200/213 - Loss: 0.0035
INFO:root:Training completed. Average Loss: 0.0159
INFO:root:Start Evaluation


Evaluating:   0%|          | 0/121 [00:00<?, ?it/s]

INFO:root:Batch 50/121 - Loss: 0.0497
INFO:root:Batch 100/121 - Loss: 0.1318
INFO:root:Evaluation completed. Average Loss per Batch: 0.1571
INFO:root:Training Loss: 0.01585480494221384 | Eval Loss: 0.1571097114428393
INFO:root:Epoch completed in 87.17566776275635 s



Token-Level Evaluation Metrics:
╒════════════════╤═════════════╤══════════╤══════════╤═══════════╕
│ Tag            │   Precision │   Recall │       F1 │   Support │
╞════════════════╪═════════════╪══════════╪══════════╪═══════════╡
│ B-LOC          │    0.895921 │ 0.953593 │ 0.923858 │       668 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ B-MISC         │    0.79397  │ 0.67234  │ 0.728111 │       235 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ B-ORG          │    0.855721 │ 0.764444 │ 0.807512 │       450 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ B-PERS         │    0.885914 │ 0.886946 │ 0.88643  │       858 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ I-LOC          │    0.816092 │ 0.855422 │ 0.835294 │        83 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ I-MISC         │    0.852273 │ 0.454545 │ 0.592885 │       165 │
├────────────────┼───────────

In [ ]:
metrics = trainer.training_loop()

INFO:root:Start Training Epoch: 1
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Training:   0%|          | 0/260 [00:00<?, ?it/s]

INFO:root:Batch 50/260 - Loss: 0.4074
INFO:root:Batch 100/260 - Loss: 0.0246
INFO:root:Batch 150/260 - Loss: 0.0614
INFO:root:Batch 200/260 - Loss: 0.0853
INFO:root:Batch 250/260 - Loss: 0.0731
INFO:root:Training completed. Average Loss: 0.3119
INFO:root:Start Evaluation
INFO:root:Test Split is Used for Evaluation


Evaluating:   0%|          | 0/121 [00:00<?, ?it/s]

INFO:root:Batch 50/121 - Loss: 0.0481
INFO:root:Batch 100/121 - Loss: 0.1016
INFO:root:Evaluation completed. Average Loss per Batch: 0.1500
INFO:root:Training Loss: 0.3118689347488376 | Eval Loss: 0.1500463832385284
INFO:root:Epoch completed in 101.97835659980774 s
INFO:root:Start Training Epoch: 2


...

Training:   0%|          | 0/260 [00:00<?, ?it/s]

INFO:root:Batch 50/260 - Loss: 0.0335
INFO:root:Batch 100/260 - Loss: 0.0105
INFO:root:Batch 150/260 - Loss: 0.0058
INFO:root:Batch 200/260 - Loss: 0.0320
INFO:root:Batch 250/260 - Loss: 0.0255
INFO:root:Training completed. Average Loss: 0.0262
INFO:root:Start Evaluation
INFO:root:Test Split is Used for Evaluation


Evaluating:   0%|          | 0/121 [00:00<?, ?it/s]

INFO:root:Batch 50/121 - Loss: 0.0509
INFO:root:Batch 100/121 - Loss: 0.1357
INFO:root:Evaluation completed. Average Loss per Batch: 0.1866
INFO:root:Training Loss: 0.026187226183985612 | Eval Loss: 0.18661319900088566
INFO:root:Epoch completed in 102.16551947593689 s



Token-Level Evaluation Metrics:
╒════════════════╤═════════════╤══════════╤══════════╤═══════════╕
│ Tag            │   Precision │   Recall │       F1 │   Support │
╞════════════════╪═════════════╪══════════╪══════════╪═══════════╡
│ B-LOC          │    0.903988 │ 0.916168 │ 0.910037 │       668 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ B-MISC         │    0.762136 │ 0.668085 │ 0.712018 │       235 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ B-ORG          │    0.85752  │ 0.722222 │ 0.784077 │       450 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ B-PERS         │    0.864155 │ 0.882284 │ 0.873126 │       858 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ I-LOC          │    0.831325 │ 0.831325 │ 0.831325 │        83 │
├────────────────┼─────────────┼──────────┼──────────┼───────────┤
│ I-MISC         │    0.782051 │ 0.369697 │ 0.502058 │       165 │
├────────────────┼───────────

In [ ]:
y_true = metrics.entity_outputs.get('y_true')
y_pred = metrics.entity_outputs.get('y_pred')

In [ ]:
from seqeval.metrics import classification_report as seq_classification
print(seq_classification(y_true = y_true, y_pred = y_pred, scheme='IOB2'))

In [ ]:
len(y_true)

830

In [ ]:
len(y_pred)

830